# README
I created this to learn Plotly using hte existing algo code. Once I make the basic plots, I will integrate with the main project.

### customising the theme and look of Jupyter notebook
#!jt -l
!jt -t gruvboxd
To restore the default theme of the notebook, run the command line statement as !jt -r

Enable the toolbar: !jt -t [theme name] -T

Enable the filename and logo: !jt -t [theme name] -N

Enable the kernel logo: !jt -t [theme name] -kl

In [43]:
# making notebook wider
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import plotly.io as pio 
# fix for amking sure plotly plots dont disappear https://plotly.com/python/troubleshooting/
pio.renderers.default='notebook'

In [44]:
import pandas as pd
import json 
import matplotlib.pyplot as plt
import seaborn as sns
import random
import datetime as dt
pd.set_option('display.max_columns', None) # so that we can view the pandas dataframes better source link
pd.set_option("display.precision", 2) # https://towardsdatascience.com/pandas-full-tutorial-on-a-single-dataset-4aa43461e1e2


#All necessary plotly libraries
import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

## Functions for getting prices and setting indicators</font>

In [45]:
from pycoingecko import CoinGeckoAPI # API docs are here: https://www.coingecko.com/en/api/documentation 
cg = CoinGeckoAPI()

In [46]:
def get_daily_prices(coin,days=3500):
    
    hist_prices_json= cg.get_coin_market_chart_by_id(id=coin, 
                                                           vs_currency="usd",days=days, interval="daily")
    hist_prices=pd.DataFrame(hist_prices_json['prices'], columns =['date','price']).merge(
    pd.DataFrame(hist_prices_json['market_caps'], columns =['date','market_cap']), on="date").merge(
    pd.DataFrame(hist_prices_json['total_volumes'], columns =['date','volume']), on="date")
    hist_prices['coin']=["btc"]*len(hist_prices.index)
    hist_prices['date'] = list (map( lambda n: int(float(n)/1000), hist_prices['date'])) # fixing date
    hist_prices['date']=pd.to_datetime(hist_prices['date'],unit='s')
    return hist_prices

In [47]:
def perc_diff (col1, col2):
    return (col1-col2)/col2

In [48]:
def calc_diff_trend (col) :
    col_diff=col+'_diff'
    col_trend=col+'_trend'
    btc_hist_prices[col_diff] = None
    btc_hist_prices[col_trend] = None
    btc_hist_prices[col_diff]=btc_hist_prices[col].diff()
    btc_hist_prices.loc[btc_hist_prices[col_diff]>0,col_trend] = 'UP'
    btc_hist_prices.loc[btc_hist_prices[col_diff]<=0,col_trend] = 'DOWN'
    btc_hist_prices.loc[((btc_hist_prices[col_trend] == 'UP') & (
        btc_hist_prices[col_trend].shift()=='DOWN')),col_trend] = 'REV_UP'
    btc_hist_prices.loc[((btc_hist_prices[col_trend] == 'DOWN') & (
        btc_hist_prices[col_trend].shift()=='UP')),col_trend] = 'REV_DOWN'

    

In [49]:
def create_indicators(coin_hist_prices):
    coin_hist_prices['price_sma_7']= coin_hist_prices['price'].rolling(7).mean()
    coin_hist_prices['price_sma_30']= coin_hist_prices['price'].rolling(30).mean()
    coin_hist_prices['price_sma_90']= coin_hist_prices['price'].rolling(90).mean()
    coin_hist_prices['price_sma_180']= coin_hist_prices['price'].rolling(180).mean()
    coin_hist_prices['vol_sma_7']= coin_hist_prices['volume'].rolling(7).mean()
    coin_hist_prices['vol_sma_30']= coin_hist_prices['volume'].rolling(30).mean()
    coin_hist_prices['vol_sma_90']= coin_hist_prices['volume'].rolling(90).mean()
    coin_hist_prices['vol_sma_180']= coin_hist_prices['volume'].rolling(180).mean()
    coin_hist_prices['dev_price_sma_7'] = perc_diff(coin_hist_prices['price'],
                                                     coin_hist_prices['price_sma_7'])
    coin_hist_prices['dev_price_sma_30'] = perc_diff(coin_hist_prices['price'],
                                                     coin_hist_prices['price_sma_30'])
    coin_hist_prices['dev_price_sma_90'] = perc_diff(coin_hist_prices['price'],
                                                     coin_hist_prices['price_sma_90'])
    coin_hist_prices['dev_price_sma_180'] = perc_diff(coin_hist_prices['price'],
                                                     coin_hist_prices['price_sma_180'])
    coin_hist_prices['dev_vol_sma_7'] = perc_diff(coin_hist_prices['volume'],
                                                   coin_hist_prices['vol_sma_7'])
    coin_hist_prices['dev_vol_sma_30'] = perc_diff(coin_hist_prices['volume'],
                                                         coin_hist_prices['vol_sma_30'])
    coin_hist_prices['dev_vol_sma_90'] = perc_diff(coin_hist_prices['volume'],
                                                         coin_hist_prices['vol_sma_90'])
    coin_hist_prices['dev_vol_sma_180'] = perc_diff(coin_hist_prices['volume'],
                                                         coin_hist_prices['vol_sma_180'])
    coin_hist_prices['dev_vol_7_30'] = perc_diff(coin_hist_prices['vol_sma_7'],
                                                     coin_hist_prices['vol_sma_30'])
    coin_hist_prices['dev_price_7_30'] = perc_diff(coin_hist_prices['price_sma_7'],
                                                     coin_hist_prices['price_sma_30'])
    calc_diff_trend ('dev_price_7_30')
    calc_diff_trend ('dev_vol_7_30')
    calc_diff_trend ('dev_price_sma_7')
    
    #FOR LATER: maybe do this with map function later? and lambda?

In [50]:
btc_hist_prices=get_daily_prices("bitcoin")
eth_hist_prices=get_daily_prices("ethereum")

In [51]:
create_indicators(btc_hist_prices)
create_indicators(eth_hist_prices)

In [52]:
# btc_hist_prices[['date','price','price_sma_30',
#                        'dev_price_7_30','dev_price_7_30_diff','dev_price_7_30_trend',
#                      'dev_vol_7_30','dev_vol_7_30_diff','dev_vol_7_30_trend' ,
#                      'dev_price_sma_7','dev_price_sma_7_diff','dev_price_sma_7_trend']].sample(10)

## <font color='purple'>Functions for calculating and generating trading signals and recos </font>

In [53]:
def isBuyorSell(coin_hist_prices,date):
    return (coin_hist_prices[coin_hist_prices.date==date]['signal'].iloc[0])

In [54]:
def create_trade(coin_hist_prices,buy_date):
    
    if(type(buy_date)==str):
        buy_date=datetime.datetime.strptime(buy_date, format)
    
    sell_date=buy_date 
    coin_hist_prices.loc[coin_hist_prices.date ==buy_date,'trade_reco'] ='BUY'
    
    for idate in btc_hist_prices.date:
        if (idate <= buy_date):
            continue # skip the rest of the loop until we are past the buy_date
        elif (isBuyorSell(coin_hist_prices,idate)=='SELL'):
            coin_hist_prices.loc[coin_hist_prices.date ==idate,'trade_reco'] ='SELL'
            sell_date=idate
            break # if SELL found, break the loop
    
    return sell_date

In [55]:
def calc_signals(coin_hist_prices):
    
    coin_hist_prices['signal'] = None 
    
    # Calculate BUY signals
    for index, row in coin_hist_prices.iterrows():
         if ((row['dev_price_7_30_trend'] =='UP') & (row['dev_vol_7_30_trend'] =='UP')):
            coin_hist_prices.at[index,'signal'] ='BUY'
    #      if ((row['dev_price_7_30'] <0)& (row['dev_price_7_30_trend']=='REV_UP')& ((row['dev_vol_7_30_trend']=='REV_UP')|(row['dev_vol_7_30_trend']=='UP'))):
    #          coin_hist_prices.at[index,'signal'] ='BUY'

    # Calculate SELL signals 
    for index, row in coin_hist_prices.iterrows():
        if (row['dev_price_7_30_trend']=='REV_DOWN'):
             coin_hist_prices.at[index,'signal'] ='SELL'
        elif ((row['dev_price_7_30'] <0)& (row['dev_price_7_30_trend']=='DOWN')):
            coin_hist_prices.at[index,'signal'] ='SELL'

    #### HAVe REVISED SELL SIGNAL to add a condition that dev price 7 30 trend != REV_UP to signal a sell when its -ve

In [56]:
def gen_trades(coin_hist_prices):
    
    coin_hist_prices.trade_reco = None
    start_date= coin_hist_prices.loc[0]['date']
    
    for idate in coin_hist_prices.date : 
        if (idate<=start_date):
            continue # skip the code below if this date was already scanned for a trade signal
        elif (isBuyorSell(coin_hist_prices,idate)=='BUY'):
            start_date=create_trade(coin_hist_prices,idate) # record the trade and scan next 
        

In [57]:
calc_signals(btc_hist_prices)

In [58]:
gen_trades(btc_hist_prices)

In [59]:
btc_hist_prices[['date','price','price_sma_30','dev_price_sma_30','signal', 'trade_reco']].sample(10)

## <font color='purple'>Plotting Funcitons</font>

In [60]:

# function to plot n number of idnicators on the same axis - DONE
# function to plot price on left axis (faded) and n-number of other indicators on right axis - DONE 
# function to show buy and sell signals for a given date range, or a given start-end range of DF 
# function to show buy and sell signals as well as n number of indicators on right axis 


In [61]:
sns.set_theme(style="white")
sns.set(rc={'figure.figsize':(15,8)})

### Couple of learning points about Python functions 
<i>**these are also noted in my onenote in python learning in methods and functions section**</i>

#### Effect on mutable vs immutable arguments
	• Source : https://stackoverflow.com/questions/575196/why-can-a-function-modify-some-arguments-as-perceived-by-the-caller-but-not-oth 
	• List, arrays, etc, will change if modified in the function body but not int, str, etc.
	

#### Named arguments vs positisonal arguments
	• Source: https://www.educative.io/answers/what-are-keyword-arguments-in-python
	• If we don’t call an arugment by its name, its called positional argument, and python will try to guess it based on the order defined in function definition
	• There needs to be an order to this
		○  With keyword arguments, as long as you assign a value to the parameter, the positions of the arguments do not matter. 
        ○ However, they do have to come after positional arguments and before default/optional arguments in a function call.

#### Passing list or dictionary as an argument 
It seems to me that only reason you don’t do this and instead use *args, and **kwargs is because they will be modified by function as they are mutable!!!

In [62]:
#plot n number of idnicators on the same axis 
def plot_inds(coin_hist_prices,inds=[],start_date='2001-01-01',end_date='2001-01-01', num_days=180 ):
    start_date = dt.datetime.strptime(start_date, '%Y-%m-%d') # https://www.digitalocean.com/community/tutorials/python-string-to-datetime-strptimea
    end_date = dt.datetime.strptime(end_date, '%Y-%m-%d')
    default_date=dt.datetime.strptime('2001-01-01', '%Y-%m-%d')
  
    # set the dates
    if (start_date==default_date) & (end_date==default_date):
            end_date=coin_hist_prices.iloc[len(coin_hist_prices)-2]['date']
            start_date=end_date - dt.timedelta(days=num_days)

    elif start_date==default_date:
        start_date= end_date - dt.timedelta(days=num_days) # https://www.geeksforgeeks.org/how-to-add-and-subtract-days-using-datetime-in-python/
    elif end_date==default_date:
        end_date=start_date + dt.timedelta(days=num_days)
    
    print('start date:',start_date,'end date:',end_date)
    for ind in inds:
        sns.lineplot(x='date', y=ind,data=coin_hist_prices.loc[((coin_hist_prices['date']>=start_date)&(coin_hist_prices['date']<=end_date)),['date',ind]])

In [63]:
# possible calls 
#plot_inds(btc_hist_prices,inds=['price','price_sma_30','price_sma_7'])
#plot_inds(btc_hist_prices,inds=['price','price_sma_30'],start_date='2020-01-01')
#plot_inds(btc_hist_prices,inds=['price','price_sma_30'],start_date='2020-01-01',num_days=500)
#plot_inds(btc_hist_prices,inds=['price','price_sma_30'],start_date='2020-01-01',end_date='2020-09-01')
#plot_inds(btc_hist_prices,inds=['price','price_sma_30'],start_date='2020-01-01')
#plot_inds(btc_hist_prices,inds=['price'],num_days=500)

In [70]:
# plot UPTO 3 indicators vs price
# LATER: add LEGEND
def plot_price_vs_inds (coin_hist_prices,vs_inds=[],start_date='2001-01-01',end_date='2001-01-01', num_days=180,colors=['red','blue','green']): # or inds_on_price?
    
    start_date = dt.datetime.strptime(start_date, '%Y-%m-%d')
    end_date = dt.datetime.strptime(end_date, '%Y-%m-%d')
    default_date=dt.datetime.strptime('2001-01-01', '%Y-%m-%d')
    
  
    # set the dates
    if (start_date==default_date) & (end_date==default_date):
            end_date=coin_hist_prices.iloc[len(coin_hist_prices)-2]['date']
            start_date=end_date - dt.timedelta(days=num_days)

    elif start_date==default_date:
        start_date= end_date - dt.timedelta(days=num_days) 
    elif end_date==default_date:
        end_date=start_date + dt.timedelta(days=num_days)
    print('start date:',start_date,'end date:',end_date)
    
    plot_data=data=coin_hist_prices.loc[((coin_hist_prices['date']>=start_date)&(coin_hist_prices['date']<=end_date))]
    
    # plot price
    sns.set_theme(style="darkgrid")

    sns.lineplot(x='date', y='price',
                     data=plot_data[['date','price']],color='black', alpha=0.2)

        # plot other inds on different axis
    ax2=plt.twinx() 
    
    for (i,ind) in enumerate(vs_inds):
        sns.lineplot(x='date', y=ind,data=plot_data[['date',ind]],ax=ax2,color=colors[i])

In [65]:
#plot_price_vs_inds(btc_hist_prices,vs_inds=['dev_price_sma_7','dev_price_7_30'],colors=['blue','green'])
#plot_price_vs_inds(btc_hist_prices,vs_inds=['dev_price_sma_7','dev_price_7_30'],colors=['blue','green'], num_days=500)
plot_price_vs_inds(btc_hist_prices,vs_inds=['dev_price_sma_7','dev_price_7_30'],start_date='2019-06-09', num_days=200)

In [66]:
# Plot buy sell recos

In [67]:
def plot_signals(coin_hist_prices,inds=[],start_date='2001-01-01',end_date='2001-01-01', num_days=180,colors=['orange','purple','grey'] ):
    
    start_date = dt.datetime.strptime(start_date, '%Y-%m-%d')
    end_date = dt.datetime.strptime(end_date, '%Y-%m-%d')
    default_date=dt.datetime.strptime('2001-01-01', '%Y-%m-%d')
    
    # set the dates
    if (start_date==default_date) & (end_date==default_date):
            end_date=coin_hist_prices.iloc[len(coin_hist_prices)-2]['date']
            start_date=end_date - dt.timedelta(days=num_days)
    elif start_date==default_date:
        start_date= end_date - dt.timedelta(days=num_days) 
    elif end_date==default_date:
        end_date=start_date + dt.timedelta(days=num_days)
    
    print('start date:',start_date,'end date:',end_date)
    
    plot_data=data=coin_hist_prices.loc[((coin_hist_prices['date']>=start_date)&(coin_hist_prices['date']<=end_date))]
    
    # plot price
    sns.set_theme(style="darkgrid")
    sns.lineplot(x='date', y='price', data=plot_data[['date','price']],color='b')
    
    # setting the x and y offset 
    (y_lim1,y_lim2)=plt.ylim()
    y_off= 0.1* (y_lim2-y_lim1)
    (x_lim1,x_lim2)=plt.xlim()
    x_off= 0.025* (x_lim2-x_lim1)

    # plot signals
    for index, row in plot_data.iterrows():
        if (row['trade_reco'] == 'BUY'):
            plt.annotate(row['trade_reco'],xy=(row['date'], row['price']),xytext=((row['date']+dt.timedelta(days=5), (row['price']-y_off))),
                         color='green', arrowprops=dict(facecolor='grey', width=2.5,headwidth=7.5))
        elif (row['trade_reco'] == 'SELL'):
            plt.annotate(row['trade_reco'],xy=(row['date'], row['price']),xytext=((row['date']+dt.timedelta(days=5), (row['price']+y_off))),
                         color='red', arrowprops=dict(facecolor='grey', width=2.5,headwidth=7.5))

    # plot other inds on different axis
    ax2=plt.twinx() 
    
    for (i,ind) in enumerate(inds):
        sns.lineplot(x='date', y=ind,data=plot_data[['date',ind]],ax=ax2,color=colors[i])


In [68]:
plot_signals(btc_hist_prices)

## <font color='purple'>Learning Plotly</font>

## <font color='red'>cant figure out how to make the y axes autoscale properly</font>

* Plotly gives us teh ability to zoom in and zoom out of charts, see data upon hover, and several other advanced charting capabilites by leveraging JS in the backend 
* Plotly express makes lives easier by providing the ability to create charts super quickly with very little code
* Plotly graph objects is the lower level implmenetation but really confugurable and verbose and is well suited for my purposes so I will focus mainly on Plotly Graph Objects 

* We basically create a graph object and then create traces (which are the series to be plotted) and update layouts and pretty much specify anything we like nad hten do fig.show()

* we can have some stuff on right hand side or add annotataions, add range sliders, etc. etc. etc. 
* Dict is the preferred way of specifying inputs and its overall very verbose becuase its quite configurable 

### Make an interactive price only chart that can zoom in on specific date ranges and shows data on hover

#### Using Plotly express (Not very useful for me)

In [69]:
fig = px.line(btc_hist_prices, x='date', y='price', title = 'BTC Prices')

fig.update_xaxes(rangeslider_visible=True, rangeselector=dict(
    buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.show()

#### trying it now using just graph objects as thats a lot more flexibile 

* great overall link on using plotly especially for finacial data and how to lot many lineso n teh same plot, etc. 
https://www.kaggle.com/code/yash161101/financial-charts-using-plotly-cheat-sheet

* changing Plotly figure size:  https://stackoverflow.com/questions/59953431/how-to-change-plotly-figure-size

* crating line charts and multiple line charts using Plotly goobjects https://lifewithdata.com/2022/02/18/line-chart-how-to-create-a-line-chart-in-plotly-python/

* Adding a range slider https://plotly.com/python/range-slider/ 

In [42]:
fig =go.Figure()
fig.add_trace(go.Scatter(x=btc_hist_prices['date'],y=btc_hist_prices['price']))
fig.update_layout(title='BTC prices',autosize=False, width=1000, height=700,margin=go.layout.Margin(
        l=50, r=50, b=100, t=100, pad = 4
    ),
        xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,label="1m", step="month", stepmode="backward"),
                dict(count=6,label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True), type="date"
    )
)

fig.show()

### Make an interactive price chart with SEVERAL INDICATORS ON SAME AXIS that can zoom in on specific date ranges and shows data on hover

#### Using Plotly express

In [37]:


fig = px.line(btc_hist_prices, x='date', y=['price','price_sma_7','price_sma_30'], title = 'BTC Prices')

fig.update_layout({'autosize':False,
    'width':1000,
    'height':800,

    'margin':go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad = 4)
})
fig.update_xaxes(rangeslider_visible=True, rangeselector=dict(
    buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
                )
fig.update_yaxes({'fixedrange':False})



fig.show()

#### Using Plotly Objects

* great overall link on using plotly especially for finacial data and how to lot many lineso n teh same plot, etc. 
https://www.kaggle.com/code/yash161101/financial-charts-using-plotly-cheat-sheet

* crating line charts and multiple line charts using Plotly goobjects https://lifewithdata.com/2022/02/18/line-chart-how-to-create-a-line-chart-in-plotly-python/

In [39]:
fig =go.Figure()
fig.add_trace(go.Scatter(x=btc_hist_prices['date'],y=btc_hist_prices['price'],name='Price',mode='lines'))
fig.add_trace(go.Scatter(x=btc_hist_prices['date'],y=btc_hist_prices['price_sma_30'],name='SMA-30',mode='lines'))
fig.update_layout(title='BTC prices',autosize=False, width=1000, height=700,margin=go.layout.Margin(
        l=50, r=50, b=100, t=100, pad = 4
    ),
        xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,label="1m", step="month", stepmode="backward"),
                dict(count=6,label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True), type="date"
    )
)
fig.show()

### Make an interactive price chart with SEVERAL INDICATORS ON different AXIS that can zoom in on specific date ranges and shows data on hover

* Great link on creaitng second axis, from plotly official website:
    https://plotly.com/python/multiple-axes/#multiple-y-axes-and-plotly-express

In [31]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=btc_hist_prices['date'],y=btc_hist_prices['price'],name='Price',mode='lines'),secondary_y=False)
fig.add_trace(go.Scatter(x=btc_hist_prices['date'],y=btc_hist_prices['price_sma_30'],name='SMA-30',mode='lines'),secondary_y=False)
fig.add_trace(go.Scatter(x=btc_hist_prices['date'],y=btc_hist_prices['dev_price_sma_30'],name='Dev-Price-SMA-30',mode='lines'),secondary_y=True)
fig.update_layout(title='BTC prices',autosize=False, width=1000, height=700,margin=go.layout.Margin(
        l=50, r=50, b=100, t=100, pad = 4
    ),
        xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,label="1m", step="month", stepmode="backward"),
                dict(count=6,label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True), type="date"
    )
)
fig.show()

### IF EASY: make a graph showing buy/sell indicators (DONE)

### IF EASY: make a graph showing buy/sell indicators as well as input signals on 2 different axes + interactive (DONE)

### IF EASY: make a graph showing buy/sell indicators as well as input signals on 2 different axes + interactive  AND I can turn signals on and off from UI (DONE)

* Adding annotations to charts: https://plotly.com/python/text-and-annotations/ 
* Adding texton chart: https://stackoverflow.com/questions/62716521/plotly-how-to-add-text-to-existing-figure

In [73]:
# plot UPTO 3 indicators vs price
# LATER: add LEGEND
def plotly_signals_and_inds (coin_hist_prices,vs_inds=[],start_date='2001-01-01',end_date='2001-01-01', num_days=180,colors=['red','blue','green']): # or inds_on_price?
    
    start_date = dt.datetime.strptime(start_date, '%Y-%m-%d')
    end_date = dt.datetime.strptime(end_date, '%Y-%m-%d')
    default_date=dt.datetime.strptime('2001-01-01', '%Y-%m-%d')
    
  
    # set the dates
    if (start_date==default_date) & (end_date==default_date):
            end_date=coin_hist_prices.iloc[len(coin_hist_prices)-2]['date']
            start_date=end_date - dt.timedelta(days=num_days)

    elif start_date==default_date:
        start_date= end_date - dt.timedelta(days=num_days) 
    elif end_date==default_date:
        end_date=start_date + dt.timedelta(days=num_days)
    print('start date:',start_date,'end date:',end_date)
    
    plot_data=data=coin_hist_prices.loc[((coin_hist_prices['date']>=start_date)&(coin_hist_prices['date']<=end_date))]
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Plot price
    fig.add_trace(go.Scatter(x=plot_data['date'],y=plot_data['price'],name='Price',mode='lines', 
                             opacity=0.2, line_color='black'),secondary_y=False)
    # changing line opacity and color
    # https://stackoverflow.com/questions/50488894/plotly-py-change-line-opacity-leave-markers-opaque
    #https://stackoverflow.com/questions/58188816/how-to-set-line-color
    
    # plot vs indicators 
    for (i,ind) in enumerate(vs_inds):
            fig.add_trace(go.Scatter(x=plot_data['date'],y=plot_data[ind],name=ind,mode='lines', 
                             line_color=colors[i]),secondary_y=True)
     # plot signals
    for index, row in plot_data.iterrows():
        if (row['trade_reco'] == 'BUY'):
            fig.add_annotation(x=row['date'], y=row['price'],text="BUY", showarrow=True, arrowhead=1)
        elif (row['trade_reco'] == 'SELL'):
            fig.add_annotation(x=row['date'], y=row['price'],text="SELL", showarrow=True, arrowhead=1)

    fig.update_layout(autosize=False, width=1200, height=700,margin=go.layout.Margin(
    l=20, r=20, b=20, t=20, pad = 1),
    xaxis=dict(rangeselector=dict( buttons=list([
            dict(count=1,label="1m", step="month", stepmode="backward"),
            dict(count=6,label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    ), rangeslider=dict(visible=True), type="date"), legend_orientation='h' )

    fig.update_yaxes({'fixedrange':False})    
    
    return fig.show()

In [72]:
plotly_signals_and_inds(btc_hist_prices,vs_inds=['dev_price_sma_7','dev_price_7_30'],start_date='2019-06-09', num_days=200)

In [ ]:
#### Minimising margins and legends

In [49]:
plotly_signals_and_inds(btc_hist_prices,vs_inds=['dev_price_sma_7','dev_price_7_30'],start_date='2019-06-09', num_days=200)

start date: 2019-06-09 00:00:00 end date: 2019-12-26 00:00:00


## IF EASY:
1. add buttons to choose range of plotting - NOT REQUIRED/OVERKILL
2. buttons to turn indicators on and off - DONE
3. better autoscaling - NOT REQUIRED
4. reduce margin at sides - DONE
5. change jupyter notebook theme - DONE 